# hp Schema Extraction

This notebook demonstrates RDF schema extraction from the hp SPARQL endpoint. It discovers VoID (Vocabulary of Interlinked Datasets) descriptions and generates  JSON-LD as the source for all downstream outputs including frequency analysis and LinkML schemas.

## Exports

- [JSON-LD Schema](https://github.com/jmillanacosta/rdfsolve/blob/main/docs/data/schema_extraction/hp_schema/hp_schema.jsonld) (primary output)
- [N-Quads RDF](https://github.com/jmillanacosta/rdfsolve/blob/main/docs/data/schema_extraction/hp_schema/hp_schema.nq)
- [VoID Graph](https://github.com/jmillanacosta/rdfsolve/blob/main/docs/data/schema_extraction/hp_schema/hp_generated_void.ttl) for the dataset in its original source
- [Coverage report](https://github.com/jmillanacosta/rdfsolve/blob/main/docs/data/schema_extraction/hp_schema/hp_pattern_coverage.csv)
- [LinkML Schema](https://github.com/jmillanacosta/rdfsolve/blob/main/docs/data/schema_extraction/hp_schema/hp_linkml_schema.yaml)
- [Full parquet entity dataframe](https://github.com/jmillanacosta/rdfsolve/blob/main/docs/data/schema_extraction/hp_schema/hpinstances.parquet)

In [ ]:
# Dataset Configuration
import os

# Dataset parameters
endpoint_url = "https://bio2rdf.org/sparql"
dataset_name = "hp"
void_iri = "http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3"
graph_uri = "http://bio2rdf.org/hp_resource:bio2rdf.dataset.hp.R3"

# Setup paths
working_path = os.path.abspath("")
exports_path = os.path.join(
    working_path, "..", "..", "docs", "data", "schema_extraction", dataset_name
)
os.makedirs(exports_path, exist_ok=True)

In [ ]:
import logging
import sys

# Minimal notebook logger using existing dataset_name
logger = logging.getLogger(dataset_name or "notebook")
logger.setLevel(logging.DEBUG)  # Set to DEBUG to see SPARQL queries

# Also configure the rdfsolve.parser logger to see query details
parser_logger = logging.getLogger("rdfsolve.parser")
parser_logger.setLevel(logging.DEBUG)

# Avoid adding duplicate handlers if the cell is re-run
if not logger.handlers:
    fmt = logging.Formatter("%(asctime)s %(levelname)s %(name)s: %(message)s", "%Y-%m-%d %H:%M:%S")

    sh = logging.StreamHandler(sys.stdout)
    sh.setLevel(logging.DEBUG)  # Set to DEBUG to see all logs
    sh.setFormatter(fmt)
    logger.addHandler(sh)

    # Add the same handler to the parser logger
    parser_logger.addHandler(sh)

logger.info(f"Logging configured for {dataset_name}")

In [ ]:
# Import libraries
import json

# Configure Plotly for HTML output
import plotly.io as pio
import plotly.offline as pyo
from IPython.display import Markdown, display

from rdfsolve.parser import VoidParser

# Set renderer to 'notebook' for Jupyter, but ensure HTML export works
pio.renderers.default = "notebook+plotly_mimetype"

# Initialize offline mode for Plotly
pyo.init_notebook_mode(connected=True)

In [ ]:
# Pickle caching utilities
import os
import pickle


def save_cache(data, filename, cache_dir=None):
    """Save data to pickle cache."""
    if cache_dir is None:
        cache_dir = os.path.join(exports_path, "cache")
    os.makedirs(cache_dir, exist_ok=True)

    cache_path = os.path.join(cache_dir, f"{filename}.pkl")
    with open(cache_path, "wb") as f:
        pickle.dump(data, f)
    print(f"Cached data to: {cache_path}")
    return cache_path


def load_cache(filename, cache_dir=None):
    """Load data from pickle cache if it exists."""
    if cache_dir is None:
        cache_dir = os.path.join(exports_path, "cache")

    cache_path = os.path.join(cache_dir, f"{filename}.pkl")
    if os.path.exists(cache_path):
        with open(cache_path, "rb") as f:
            data = pickle.load(f)
        print(f"Loaded cached data from: {cache_path}")
        return data
    return None


def cache_exists(filename, cache_dir=None):
    """Check if cache file exists."""
    if cache_dir is None:
        cache_dir = os.path.join(exports_path, "cache")

    cache_path = os.path.join(cache_dir, f"{filename}.pkl")
    return os.path.exists(cache_path)

In [ ]:
# Cache management utilities
def list_cache_files(cache_dir=None):
    """List all cache files."""
    if cache_dir is None:
        cache_dir = os.path.join(exports_path, "cache")

    if not os.path.exists(cache_dir):
        print("No cache directory found")
        return []

    cache_files = [f for f in os.listdir(cache_dir) if f.endswith(".pkl")]
    print(f"Cache directory: {cache_dir}")
    for f in cache_files:
        file_path = os.path.join(cache_dir, f)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"  {f} ({size_mb:.2f} MB)")
    return cache_files


def clear_cache(filename=None, cache_dir=None):
    """Clear specific cache file or all cache."""
    if cache_dir is None:
        cache_dir = os.path.join(exports_path, "cache")

    if filename:
        cache_path = os.path.join(cache_dir, f"{filename}.pkl")
        if os.path.exists(cache_path):
            os.remove(cache_path)
            print(f"Removed cache: {filename}")
        else:
            print(f"Cache not found: {filename}")
    else:
        # Clear all cache files
        if os.path.exists(cache_dir):
            import shutil

            shutil.rmtree(cache_dir)
            print("Cleared all cache files")
        else:
            print("No cache directory to clear")


# Show current cache status
list_cache_files()

### Cache Control

Use these cells to manage cached data. When testing new code changes, you may want to clear relevant cache files to force re-computation.

In [ ]:
# Clear specific cache files (uncomment lines as needed for testing)

# When testing new parser changes:
# clear_cache(f"{dataset_name}_voidparser")

# When testing JSON-LD generation (primary output):
# clear_cache(f"{dataset_name}_jsonld_schema")

# When testing frequency calculations:
# clear_cache(f"{dataset_name}_frequencies_basic")
# clear_cache(f"{dataset_name}_frequencies_with_instances")

# Clear everything:
# clear_cache()

print("Cleared parser and JSON-LD caches to test schema pattern fix")
print("Note: JSON-LD cache is now the primary cache - clear it when testing schema changes.")

## Discover or get VoID Schema

In [ ]:
# Generate VoID schema with caching
cache_key = f"{dataset_name}_voidparser"

# Try to load from cache first
vp = load_cache(cache_key)

if vp is None:
    print("VoidParser not found in cache, generating...")
    vp = VoidParser.from_endpoint_with_discovery(
        endpoint_url=endpoint_url,
        dataset_name=dataset_name,
        exports_path=exports_path,
        # Arguments for the case when a suitable VoID is not found
        graph_uris=[graph_uri] if graph_uri else None,
        exclude_graph_patterns=[
            "openlinksw",
            "well-known",
            "void",
            "service",
        ],  # Filter out service description and administrative graphs
        counts=True,
        offset_limit_steps=300,  # pagination
    )
    # Cache the VoidParser for future use
    save_cache(vp, cache_key)
else:
    print("Loaded VoidParser from cache")

## Schema Discovery and Exports Workflow

### Workflow Steps:

1. **VoID Discovery**: Extract schema patterns from SPARQL endpoint VoID descriptions
2. **JSON-LD Generation**: Convert to JSON-LD.
3. **Derived Outputs**: All other formats are generated from the JSON-LD structure:
   - **Frequencies**: Schema pattern coverage analysis
   - **LinkML**: LinkML YAML used elsewhere for other features.
   - **CSV/JSON**: Tabular and structured data exports
   - **RDF**: N-Quads serialization for triplestore import

In [ ]:
# Primary JSON-LD schema export and basic summary
cache_key = f"{dataset_name}_jsonld_schema"
jsonld_schema = load_cache(cache_key)

if jsonld_schema is None:
    print("Generating standards-compliant JSON-LD schema...")
    jsonld_schema = vp.to_jsonld(filter_void_admin_nodes=True)
    save_cache(jsonld_schema, cache_key)
else:
    print("Loaded JSON-LD schema from cache")

# Save JSON-LD schema file
jsonld_file = os.path.join(exports_path, f"{dataset_name}_schema.jsonld")
with open(jsonld_file, "w", encoding="utf-8") as f:
    json.dump(jsonld_schema, f, indent=2, ensure_ascii=False)

print(f"JSON-LD Schema saved to: {jsonld_file}")

# Display combined JSON-LD structure info and schema summary
if "@graph" in jsonld_schema:
    print("\nSchema Summary:")
    print(f"   • Prefixes: {len(jsonld_schema['@context'])}")
    print(f"   • Resources: {len(jsonld_schema['@graph'])}")

    # Show dataset metadata
    dataset_info = jsonld_schema["@graph"][0] if jsonld_schema["@graph"] else {}
    if dataset_info.get("@type") == "void:Dataset":
        print(f"   • Dataset: {dataset_info.get('dcterms:title', 'Unknown')}")
        print(f"   • Classes: {dataset_info.get('void:classes', 0)}")
        print(f"   • Properties: {dataset_info.get('void:properties', 0)}")
        print(f"   • Triples: {dataset_info.get('void:triples', 0)}")

# Get schema DataFrame and show sample
schema_df = vp.to_schema(filter_void_admin_nodes=True)
print(f"\nSchema Patterns Preview ({len(schema_df)} total):")
display(schema_df.head())

## Schema Pattern Coverage Analysis
Calculate coverage ratios showing what percentage of entities use each relationship pattern.

In [ ]:
# Schema pattern coverage analysis and export
cache_key = f"{dataset_name}_frequencies_basic"
cached_data = load_cache(cache_key)

if cached_data is None:
    print("Calculating schema pattern frequencies...")
    frequencies_df, _ = vp.count_schema_shape_frequencies(
        endpoint_url=endpoint_url,
        offset_limit_steps=300,
    )
    save_cache(frequencies_df, cache_key)
else:
    print("Loaded frequencies DataFrame from cache")
    frequencies_df = cached_data

# Export coverage analysis
frequencies_output_path = os.path.join(exports_path, f"{dataset_name}_pattern_coverage.csv")
exported_df = vp.export_schema_shape_frequencies(
    frequencies_df, output_file=frequencies_output_path
)

# Combined summary and sample
if not frequencies_df.empty:
    avg_coverage = frequencies_df["coverage_percent"].mean()
    high_coverage = (frequencies_df["coverage_percent"] > 50).sum()

    print("\nPattern Coverage Analysis:")
    print(f"   • Total patterns: {len(frequencies_df)}")
    print(f"   • Average coverage: {avg_coverage:.1f}%")
    print(f"   • High coverage (>50%): {high_coverage}")
    print(f"   • Exported to: {frequencies_output_path}")

    print("\nSample Coverage Data:")
    display(
        frequencies_df[["subject_class", "property", "object_class", "coverage_percent"]].head()
    )

    print("\nCoverage Statistics:")
    display(frequencies_df["coverage_percent"].describe())
else:
    print("No frequency data available")

## Schema Pattern Instance Collection
Collect actual subject and object IRI instances for each schema pattern. This provides detailed access to the specific entities participating in each relationship pattern.

In [ ]:
# Collect both frequency data and actual instances with caching
cache_key = f"{dataset_name}_frequencies_with_instances"
cached_data = load_cache(cache_key)

if cached_data is None:
    print("Collecting frequency data and instances...")
    frequencies_with_instances_df, instances_df = vp.count_schema_shape_frequencies(
        endpoint_url=endpoint_url,
        # sample_limit=100,  # Limited sample for demonstration
        collect_instances=True,
        offset_limit_steps=300,
    )
    # Cache both DataFrames as a tuple
    save_cache((frequencies_with_instances_df, instances_df), cache_key)
else:
    print("Loaded frequencies and instances DataFrames from cache")
    frequencies_with_instances_df, instances_df = cached_data

# Display basic information about the data structure
print(f"Frequencies DataFrame: {len(frequencies_with_instances_df)} shapes")
if instances_df is not None:
    print(f"Instances DataFrame: {len(instances_df)} subject-object pairs")
    print(
        f"Memory usage - Frequencies: {frequencies_with_instances_df.memory_usage(deep=True).sum() / 1024:.1f} KB"
    )
    print(f"Memory usage - Instances: {instances_df.memory_usage(deep=True).sum() / 1024:.1f} KB")
else:
    print("No instances collected")

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

if not frequencies_df.empty:
    df = frequencies_df.copy()
    df["coverage_percent"] = pd.to_numeric(df["coverage_percent"], errors="coerce").fillna(0)
    df = df.sort_values("coverage_percent", ascending=False).reset_index(drop=True)

    def make_label(row):
        return (
            f"<b>{row['subject_class']}</b> "
            f"<span style='color:#888;'></span> "
            f"<i>{row['property']}</i> "
            f"<span style='color:#888;'></span> "
            f"<b>{row['object_class']}</b>"
        )

    df["styled_label"] = df.apply(make_label, axis=1)

    text_positions = ["outside" if v < 95 else "inside" for v in df["coverage_percent"]]
    custom_colorscale = [
        [0.0, "#d36e61"],
        [0.4, "#e5cdbd"],
        [0.7, "#e8e4cf"],
        [1.0, "#c3d9c0"],
    ]

    # Figure sizing
    bar_height = 26
    fig_height = min(2000, bar_height * len(df) + 200)

    fig = go.Figure(
        go.Bar(
            x=df["coverage_percent"],
            y=df["styled_label"],
            orientation="h",
            text=[f"{v:.1f}%" for v in df["coverage_percent"]],
            textposition=text_positions,
            marker={
                "color": df["coverage_percent"],
                "colorscale": custom_colorscale,
                "cmin": 0,
                "cmax": 100,
                "line": {"color": "white", "width": 0.6},
            },
            hovertemplate="<b>%{y}</b><br>Coverage: %{x:.1f}%<extra></extra>",
        )
    )

    fig.update_layout(
        title={
            "text": f"Schema Pattern Coverage for {dataset_name}",
            "x": 0.5,
            "font": {"size": 18},
        },
        xaxis={
            "title": "Coverage (%)",
            "range": [0, 100],  # fixed x-axis range
            "ticksuffix": "%",
            "showgrid": True,
            "gridcolor": "rgba(220,220,220,0.3)",
        },
        yaxis={
            "title": "",
            "autorange": "reversed",
            "automargin": True,
            "fixedrange": False,  # allow vertical zoom/pan
        },
        template="plotly_white",
        autosize=True,  # allow figure to scale with container
        height=fig_height,  # base height (will scale)
        margin={"t": 80, "b": 50, "l": 480, "r": 150},  # extra right margin for text
        plot_bgcolor="white",
        paper_bgcolor="white",
    )

    # Disable horizontal zoom/pan
    fig.update_xaxes(fixedrange=True)

    # Show figure with config for HTML export compatibility
    fig.show(
        config={
            "scrollZoom": True,
            "responsive": True,
            "toImageButtonOptions": {
                "format": "png",
                "filename": f"{dataset_name}_schema_coverage",
                "height": fig_height,
                "width": 600,
                "scale": 1,
            },
        }
    )

else:
    display(Markdown("**No coverage data to visualize**"))

## LinkML (derived from JSON-LD)

In [ ]:
# Generate LinkML directly from JSON-LD with custom schema URI
print("Regenerating LinkML schema from JSON-LD with custom schema URI...")

schema_name = f"{dataset_name}_schema"
custom_schema_uri = (
    f"http://jmillanacosta.github.io/rdfsolve/{dataset_name}/linkml"  # User-definable base URI
)

yaml_text = vp.to_linkml_yaml(
    schema_name=schema_name,
    schema_description=f"LinkML schema for {dataset_name} generated from JSON-LD",
    schema_base_uri=custom_schema_uri,
    filter_void_nodes=True,
)

# Save to LinkML YAML
linkml_file = os.path.join(exports_path, f"{dataset_name}_linkml_schema.yaml")
with open(linkml_file, "w", encoding="utf-8") as f:
    f.write(yaml_text)

print(f"LinkML YAML saved to: {linkml_file}")

In [ ]:
from linkml.generators.erdiagramgen import ERDiagramGenerator
from linkml_runtime.utils.schemaview import SchemaView

sv = SchemaView(linkml_file)
linkml_schema = sv.schema

display(
    Markdown(
        f"**Parsed LinkML schema:** Classes = {len(sv.all_classes())}, Slots = {len(sv.all_slots())}"
    )
)

# Build and display a Mermaid class diagram for the aopwikirdf LinkedML
mermaid_code = ERDiagramGenerator(linkml_file).serialize()

display(Markdown(mermaid_code))

In [ ]:
json_path = os.path.join(exports_path, f"{dataset_name}_schema.json")
csv_path = os.path.join(exports_path, f"{dataset_name}_schema.csv")

# Export CSV from frequencies
frequencies_df.to_csv(csv_path, index=False)

# Export JSON derived from JSON-LD (maintains consistency)
with open(json_path, "w", encoding="utf-8") as fh:
    json.dump(vp.to_json(filter_void_nodes=True), fh, indent=2)

print(f"CSV exported to: {csv_path}")
print(f"JSON exported to: {json_path}")